In [10]:
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
import subprocess
import gc
import pandas as pd
import json
from icecream import ic
import os
from tqdm import tqdm
import time

In [ ]:
# Ones installed on my pc
model_names = [
    "llama3.1",
    "olmo-3:7b-instruct",
    "olmo-3",        # Cannot disable the thinking here but it's essentially the preivous one
    "granite3.3",
    "ministral-3",
    "qwen3",
    "qwen2.5-coder",
    "deepseek-r1",   # here with no thinking
    "deepseek-r1",
    "gemma3",
    "phi4-mini",
]

model_thinking = [
    False,
    False,
    True,
    False,
    False,
    False,
    False,
    True,
    False,
    False,
    False,
]

In [12]:
SYSTEM_PROMPT = (
    """
    You are an expert NLU annotator. Your job is to rate how plausible a candidate meaning (sense)
    is for the HOMONYM used in the target sentence within the short story.

    Return ONLY a single JSON object with one key: "score" and an integer value 1, 2, 3, 4 or 5.
    Integer mapping:
      1 = Definitely not
      2 = Probably not
      3 = Ambiguous / Unsure
      4 = Probably yes
      5 = Definitely yes

    The response must be a JSON object and nothing else, for example: {{"score": 4}}

    [EXAMPLES]
    {few_shot_examples}
    """
)

USER_PROMPT = (
    """
    Now, label this new instance:

    [STORY]
    {full_story_text}

    [HOMONYM]
    {homonym}

    [CANDIDATE SENSE]
    {sense_text}

    [TASK]
    Based on the STORY above, decide how plausible it is that the HOMONYM is used with the
    CANDIDATE SENSE in the target sentence.

    Return ONLY a single JSON object with one key "score" and an integer value (1-5)
    as described by the system message. Example output: {{"score": 3}}
    """
)

def build_few_shot_examples(df):
    """Build a five-shot examples string (one example per score 1-5) from `df`.

    Uses rounded average to determine the representative score for an item.
    """
    examples = []
    used_ids = set()

    for score in range(1, 6):
        found = False
        for idx, row in df.iterrows():
            if str(idx) in used_ids:
                continue
            try:
                avg = float(row.get("average", 0))
            except Exception:
                avg = 0.0
            if round(avg) == score:
                full = create_full_story_text(row)
                sense = f"{row.get('judged_meaning', '')} as in \"{row.get('example_sentence','')}\""
                ex = (
                    "[STORY]\n"
                    f"{full}\n\n"
                    "[HOMONYM]\n"
                    f"{row.get('homonym', '')}\n\n"
                    "[CANDIDATE SENSE]\n"
                    f"{sense}\n\n"
                    "[EXAMPLE OUTPUT]\n"
                    f'{{"score": {score}}}'
                )
                examples.append(ex)
                used_ids.add(str(idx))
                found = True
                break
        if not found:
            # fallback: include a short template example if no example found for this score
            examples.append(f"[EXAMPLE OUTPUT]\n{{\"score\": {score}}}")

    return "\n\n".join(examples)

def create_full_story_text(item):
    """Compose the story text used as context for rating.

    Uses `precontext`, `sentence`, and `ending` fields when available and joins them into a single string.
    """
    fullstory = f"{item.get('precontext', '')} {item.get('sentence', '')} {item.get('ending', '')}"
    return fullstory.strip()



In [13]:
TRAIN_JSON_FILE = "../data/train.json"
DEV_JSON_FILE = "../data/dev.json"

def load_data(file_path):
    """
    Loads the json containing the dataset and return a pandas dataframe.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    # Transpose because the json is {id: {features...}, ...}
    df = pd.DataFrame(data).T
    # Ensure 'average' is float
    df['average'] = df['average'].astype(float)
    # Ensure 'choices' is list (for scoring later)
    return df

df_train = load_data(TRAIN_JSON_FILE)
df_dev = load_data(DEV_JSON_FILE)

In [14]:
# Build and expose the few-shot examples string used by prompts
few_shot_examples = build_few_shot_examples(df_train)
ic(few_shot_examples)

def create_message(item):
    sense = f"{item.get('judged_meaning', '')} as in \"{item.get('example_sentence', '')}\"".strip()
    homonym = item.get("homonym", "")
    full_story_text = create_full_story_text(item)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT.format(
            few_shot_examples = few_shot_examples
        )},
        {"role": "user", "content": USER_PROMPT.format(
            full_story_text=full_story_text,
            homonym=homonym,
            sense_text=sense
        )},
    ]
    return messages

ic| few_shot_examples: '''[STORY]
                        The troops were stationed at a remote camp for training. To pass the time, they often engaged in recreational sports. One afternoon, they decided to play a game on the field next to their tents. The soldiers were playing a bat and ball game, which involved them running towards the base. It didn't take long for the winning team to get a home run.
                        
                        [HOMONYM]
                        base
                        
                        [CANDIDATE SENSE]
                        installation from which a military force initiates operations as in "The troops strategized at their forward base."
                        
                        [EXAMPLE OUTPUT]
                        {"score": 1}
                        
                        [STORY]
                        The old machine hummed in the corner of the workshop. Clara examined its dusty dials with a furrowed brow. She wond

In [15]:
class Score(BaseModel):
    score: int

In [16]:
# Random element in the dataset
item = df_train.sample(1).iloc[0].to_dict()

messages = create_message(item)

model_number = 6  # change to try different models

response: ChatResponse = chat(model=model_names[model_number],
                              messages=messages,
                              think=False,
                              format=Score.model_json_schema(),
                              options={
                                  "temperature": 0
                              }
                              )

# ic(messages)
ic(response.model)
ic(response.total_duration * 10e-9)  # convert from ns to s
ic(response.message.role)
ic(response.message.content)
ic(response.message.thinking)
pass

IndexError: list index out of range

In [17]:
def get_dev_predictions(model_name, df, max_examples=None, think=False):
    preds = []
    failed_ids = []
    ids = list(df.index.astype(str))
    if max_examples is not None:
        ids = ids[:max_examples]

    total_start = time.perf_counter()
    per_item_times = []

    for idx in tqdm(ids):
        item = df.loc[idx].to_dict()
        messages = create_message(item)

        start = time.perf_counter()
        try:
            response: ChatResponse = chat(model=model_name,
                                          messages=messages,
                                          think=think,          # No thinking because otherwise takes ~ 6hours for the whole dev dataset per thinking model
                                          format=Score.model_json_schema(),
                                          options={
                                              "temperature": 0
                                          }
                                          )
            elapsed = time.perf_counter() - start

            per_item_times.append((idx, elapsed))

            content = response.message.content
            try:
                s = Score.model_validate_json(content)
                pred = int(s.score)
                if pred < 1 or pred > 5:
                    raise ValueError("score out of range")
            except Exception:
                # Keep id of failed element so it can be removed from evaluation
                print("Invalid JSON or missing/invalid score for item id:", idx, "content:", content)
                pred = None
                failed_ids.append(str(idx))

        except Exception as e:
            elapsed = time.perf_counter() - start
            print(f"Error calling model {model_name} for id {idx}: {e}")
            pred = None
            failed_ids.append(str(idx))
            per_item_times.append((idx, elapsed))

        preds.append({"id": str(idx), "prediction": pred, "time": elapsed})

    total_elapsed = time.perf_counter() - total_start
    # attach summary timings as metadata and failed ids
    return {
        "predictions": preds,
        "failed_ids": failed_ids,
        "total_time": total_elapsed,
        "per_item_times": per_item_times,
        "avg_time": sum(t for _, t in per_item_times) / len(per_item_times) if per_item_times else 0,
    }

In [18]:
for model_name, think in zip(model_names, model_thinking):
    if think:
        MAX_EXAMPLES = 100  # set to an int to limit samples per model
    else:
        MAX_EXAMPLES = None  # set to None for not thinking models

    print(f"\n=== Running model: {model_name} ===")

    # If deepseek check if thinking to have different directoies
    if think:
        OUT_DIR = f"../llm-ollama/five-shot/{model_name.replace(':', '-')}-think"
        os.makedirs(OUT_DIR, exist_ok=True)
    else:
        OUT_DIR = f"../llm-ollama/five-shot/{model_name.replace(':', '-')}"
        os.makedirs(OUT_DIR, exist_ok=True)

    # get predictions (may take a while if MAX_EXAMPLES is None)
    # res = get_dev_predictions(model_name, df_dev, max_examples=MAX_EXAMPLES)
    res = get_dev_predictions(model_name, df_dev, max_examples=MAX_EXAMPLES, think=think)

    preds = res["predictions"]

    pred_file = os.path.join(OUT_DIR, "predictions.jsonl")
    with open(pred_file, "w") as f:
        for p in preds:
            f.write(json.dumps({"id": p["id"], "prediction": p["prediction"]}) + "\n")

    # Save failed ids so they can be excluded from scoring
    failed_file = os.path.join(OUT_DIR, "failed_ids.jsonl")
    with open(failed_file, "w") as f:
        for fid in res.get("failed_ids", []):
            f.write(json.dumps({"id": fid}) + "\n")

    # Save timing info and per-item times
    timing_file = os.path.join(OUT_DIR, "timing.txt")
    with open(timing_file, "w") as f:
        f.write(f"total_time_sec: {res['total_time']:.4f}\n")
        f.write(f"avg_time_sec: {res['avg_time']:.4f}\n")
        f.write("per_item_times_sec:\n")
        for idx, t in res["per_item_times"]:
            f.write(f"{idx}: {t:.4f}\n")

    # Create ref.jsonl from df_dev (respect MAX_EXAMPLES) inside the model folder
    # Exclude any ids that failed JSON parsing so they won't be evaluated
    failed_set = set(res.get("failed_ids", []))
    ref_file = os.path.join(OUT_DIR, "ref.jsonl")
    with open(ref_file, "w") as f:
        for idx, row in df_dev.iterrows():
            if MAX_EXAMPLES is not None and int(idx) >= MAX_EXAMPLES:
                break
            if str(idx) in failed_set:
                # skip items that produced invalid JSON for this model
                continue
            f.write(json.dumps({"id": str(idx), "label": row["choices"]}) + "\n")

    print(f"Predictions saved to {pred_file}")
    print(f"Gold data saved to {ref_file}")
    print(f"Timing saved to {timing_file}")

    # If there is failed attemp rewrite all of the ids so that they are consecutive. This is needed for the scoring script
    if len(res.get('failed_ids', [])) > 0:
        # Rewrite pred_file with consecutive ids
        new_pred_file = os.path.join(OUT_DIR, "predictions_consecutive_ids.jsonl")
        with open(pred_file, "r") as fin, open(new_pred_file, "w") as fout:
            for new_id, line in enumerate(fin):
                obj = json.loads(line)
                obj["id"] = str(new_id)
                fout.write(json.dumps(obj) + "\n")
        pred_file = new_pred_file

        # Rewrite ref_file with consecutive ids
        new_ref_file = os.path.join(OUT_DIR, "ref_consecutive_ids.jsonl")
        with open(ref_file, "r") as fin, open(new_ref_file, "w") as fout:
            for new_id, line in enumerate(fin):
                obj = json.loads(line)
                obj["id"] = str(new_id)
                fout.write(json.dumps(obj) + "\n")
        ref_file = new_ref_file
        
    # Run scoring script for this model outputs
    res = subprocess.run(["python", "../score/scoring.py", ref_file, pred_file, os.path.join(OUT_DIR, "score.json")], capture_output=True, text=True)
    print(res.stdout)
    if res.stderr:
        print("Scoring STDERR:")
        print(res.stderr)

    # 
    subprocess.run(["ollama", "stop", model_name], check=False)
    gc.collect()


=== Running model: smollm ===


100%|██████████| 588/588 [02:10<00:00,  4.51it/s]


Predictions saved to ../llm-ollama/five-shot/smollm/predictions.jsonl
Gold data saved to ../llm-ollama/five-shot/smollm/ref.jsonl
Timing saved to ../llm-ollama/five-shot/smollm/timing.txt
Importing...
Starting Scoring script...
Everything looks OK. Evaluating file ../llm-ollama/five-shot/smollm/predictions.jsonl on ../llm-ollama/five-shot/smollm/ref.jsonl
----------
Spearman Correlation: 0.050198506980056797
Spearman p-Value: 0.22420394764592197
----------
Accuracy: 0.5238095238095238 (308/588)
Results dumped into scores.json successfully.

